In [1]:
# Importações

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn import svm
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, confusion_matrix

import pandas as pd
import numpy as np

from nltk.corpus import stopwords

import re

import itertools

# Cross validate

from sklearn.model_selection import cross_validate

# Modelos

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
# Importação do Dataframe contendo títulos de notícias relacionadas a saúde.

dataframe = pd.read_csv('combined_df.zip')

display(dataframe)

,title,link,date,Site,noticia_falsa,corpo_texto
0,Universidade Yale publicou estudo ontem dá hid...,https://www.boatos.org/saude/universidade-yale...,10/10/2020,NaN,0,sair … o esperar meta-análise chegar o trabalh...
1,Empresa mídia interrompe impressão jornais Aus...,https://noticias.uol.com.br/internacional/ulti...,14/04/2020 09h53,NaN,1,responsável jornal impresso comunitário o aust...
2,Enfermeiro mata namorada médica acusá-la passa...,https://noticias.uol.com.br/internacional/ulti...,02/04/2020 09h34,NaN,1,o italiano antonio pace assumir matar o namora...
3,vacinação viajantes,https://www.gov.br/saude/pt-br/vacinacao/viaja...,Data não disponível,Gov - Fake,0,"['erro', 'carregar', 'texto']"
4,Contágio classes C D São Paulo 5 vezes maior A B,https://noticias.uol.com.br/saude/ultimas-noti...,17/09/2020 12h25Atualizada em 17/09/2020 17h19,NaN,1,paulistano classe c e d riscar infecção corona...
...,...,...,...,...,...,...
5140,Ministro Saúde visita hospitais Manaus,https://www.gov.br/saude/pt-br/assuntos/notici...,04/05/2020 23h37,NaN,1,n segunda-feira profissional saudar contratar ...
5141,'Estamos exaustos': efeito quarentena longa mu...,https://noticias.uol.com.br/ultimas-noticias/b...,24/08/2020 13h44,NaN,1,argentino o chamar quareterna piar o isolament...
5142,vigilância coberturas vacinação: metodologia d...,https://www.gov.br/saude/pt-br/vacinacao/publi...,Data não disponível,gov.br saude,1,"['vigilância', 'coberturas', 'vacinação', 'met..."
5143,Indígenas marcham defesa líder Yaku Pérez Equador,https://noticias.uol.com.br/ultimas-noticias/a...,11/02/2021 17h17,NaN,1,indígena equatoriano mobilizar n quinta-feira ...


In [3]:
# Cria dataframe para processamento
df = dataframe.copy()

# Transformar a coluna título em string
df['title'] = df['title'].astype(str)

In [4]:
# Variáveis para padronização/configuração

scores = ['accuracy', 'precision', 'roc_auc']
usar_links = False

In [5]:
# Checar se há títulos vazios
non_empty_titles = df['title'].apply(lambda x: len(x.strip()) > 0)
non_empty_links = df['link'].apply(lambda x: len(x.strip()) > 0)

# Checar o resultado
print(non_empty_titles.head())
print("Há documentos vazios?", not non_empty_titles.all())
print("Há links vazios?", not non_empty_links.all())
print("Número de documentos vazios:", (~non_empty_titles).sum())
print("Número de links vazios:", (~non_empty_links).sum())

0    True
1    True
2    True
3    True
4    True
Name: title, dtype: bool
Há documentos vazios? False
Há links vazios? False
Número de documentos vazios: 0
Número de links vazios: 0


In [6]:
# Iniciar o processo de TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Nesta célula, será aplicado o TF-IDF aos títulos das notícias, caso não haja nenhum vazio
temp_tit = df['title'].tolist()
if non_empty_titles.all():
    titles_tfidf = tfidf_vectorizer.fit_transform(temp_tit)
    print('TF-IDF aplicado com sucesso.')
    tfidf = titles_tfidf
else:
    print('Há documentos vazios ou apenas stop words.')

TF-IDF aplicado com sucesso.


In [7]:
def split_and_clean(link):
# Dividir o link por '/', '-', '%', '.', e ':'
    words = re.split(r'[/\-\%\.\:\_]', link)
    # Remover palavras indesejadas, stopwords e letras isoladas, mantendo acentos
    unwanted = {'https', 'www', 'com', 'br', 'pt', 'html', 'htm', 'ghtml', 'php'}
    cleaned_words = [
    word for word in words
    if not any(unwanted_word in word.lower() for unwanted_word in unwanted)
    and len(word) > 1
    ]
    return cleaned_words

if usar_links:

    # Tratando os links
    df['processed_links'] = df['link'].apply(split_and_clean)

    # TF-IDF para os combinação título e link
    temp_link = [' '.join(i) for i in df['processed_links']]
    temp_tit_link = [' '.join(list(a)) for a in zip(temp_tit, temp_link)]
    print(temp_tit_link)
    if non_empty_links.all():
        tit_link_tfidf = tfidf_vectorizer.fit_transform(temp_tit_link)
        print('TF-IDF aplicado com sucesso.')
        tfidf = tit_link_tfidf
    else:
        print('Há items vazios ou apenas stop words.')

In [8]:
# Criação de vetores
X = tfidf
y = df['noticia_falsa']

titulos = df['title'].tolist()

print("Dimensões de X:", X.shape)
print("Dimensões de y:", y.shape)

Dimensões de X: (5145, 8703)
Dimensões de y: (5145,)


## Modelos de Classificação
### Árvore de Decisão

In [9]:
# Padronizando scores para todos os modelos.

scores = ['accuracy', 'precision', 'roc_auc']

In [10]:
# Cria modelo
modelo_ad = DecisionTreeClassifier()
modelo_ad

# Validação crucru
resultados_ad = cross_validate(modelo_ad, X, y, scoring=scores)

acc_ad = resultados_ad['test_accuracy'].mean()
prec_ad = resultados_ad['test_precision'].mean()
rocauc_ad = resultados_ad['test_roc_auc'].mean()
print(f'Acurácia: {acc_ad:.3f}\nPrecisão: {prec_ad:.3f}\nROC-AUC: {rocauc_ad:.3f}')

Acurácia: 0.858
Precisão: 0.926
ROC-AUC: 0.871


### Floresta Aleatória

In [11]:
# Cria modelo
modelo_rf = RandomForestClassifier()
modelo_rf

# Validação crucru
resultados_rf = cross_validate(modelo_rf, X, y, scoring=scores)

acc_rf = resultados_rf['test_accuracy'].mean()
prec_rf = resultados_rf['test_precision'].mean()
rocauc_rf = resultados_rf['test_roc_auc'].mean()
print(f'Acurácia: {acc_rf:.3f}\nPrecisão: {prec_rf:.3f}\nROC-AUC: {rocauc_rf:.3f}')

Acurácia: 0.902
Precisão: 0.912
ROC-AUC: 0.943


### Modelo Regressão Logística

In [12]:
# Cria modelo
modelo_lr = LogisticRegression()
modelo_lr

# Validação crucru
resultados_lr = cross_validate(modelo_lr, X, y, scoring=scores)

acc_lr = resultados_lr['test_accuracy'].mean()
prec_lr = resultados_lr['test_precision'].mean()
rocauc_lr = resultados_lr['test_roc_auc'].mean()
print(f'Acurácia: {acc_lr:.3f}\nPrecisão: {prec_lr:.3f}\nROC-AUC: {rocauc_lr:.3f}')

Acurácia: 0.868
Precisão: 0.847
ROC-AUC: 0.954


### SVC

In [13]:
# Parâmetros
C = 77
kernel = 'rbf'
gamma = 0.01

# Cria modelo
modelo_svm = svm.SVC(C=C, kernel=kernel, gamma=gamma)

# Validação crucru
resultados_svm = cross_validate(modelo_svm, X, y, scoring=scores)

acc_svm = resultados_svm['test_accuracy'].mean()
prec_svm = resultados_svm['test_precision'].mean()
rocauc_svm = resultados_svm['test_roc_auc'].mean()
print(f'Acurácia: {acc_svm:.3f}\nPrecisão: {prec_svm:.3f}\nROC-AUC: {rocauc_svm:.3f}')

Acurácia: 0.914
Precisão: 0.909
ROC-AUC: 0.954


## Predição teste com dados externos

In [14]:
df_externo = pd.read_csv('Notícias Fake e Verdadeiras.txt', header=None, sep='\t', on_bad_lines='skip')
df_externo

,0,1,2
0,1,Vacinas contra a Covid-19 são mais perigosas q...,0
1,2,Vacina tem como objetivo matar seres humanos,0
2,3,Profissionais de saúde morrem por ataque cardí...,0
3,4,Butantan não confirma eficácia da CoronaVac em...,0
4,5,Vacinas de RNA mensageiro vão provocar morte e...,0
5,6,A vacina contra a Covid-19 vai modificar o DNA...,0
6,7,A vacina contra a Covid-19 tem chip líquido e ...,0
7,8,Imunizantes contra Covid-19 estão relacionados...,0
8,9,Vacinas contra Covid-19 criam campo magnético ...,0
9,10,CoronaVac não tem comprovação científica,0


In [15]:
stop_words = set(stopwords.words('portuguese'))

def trata_padrao(titulo, usar_links=False, link=None):
    '''
    Função para padronizar o tratamento dos títulos.

    Args:
    titulo (str) = título da notícia
    usar_links (bool) = usar ou não o link
    link (str) = link da notícia
    '''

    # Pré-processamento do titulo

    titulo = str(titulo).lower()
    titulo = nltk.word_tokenize(titulo)
    titulo = [word for word in titulo if word not in stop_words]
    titulo = [' '.join(titulo)]
    temp_tit = titulo

    # se usar link, pre processar link
    if usar_links:
        link = [split_and_clean(link)]
        temp_link = [' '.join(i) for i in link]
        print(temp_tit,temp_link)
        temp_tit_link = [' '.join(list(a)) for a in zip(temp_tit, temp_link)]
        processed = temp_tit_link
    else:
        processed = titulo

    return processed

In [16]:
# Define modelos para teste
modelos = [modelo_ad, modelo_rf, modelo_lr, modelo_svm]

# Treina modelos com todos os dados
for modelo in modelos:
    modelo.fit(X, y)

# Função para predizer novos títulos
def predict_title(titulo):
    preprocessed_title = trata_padrao(titulo)

    tfidf_vector = tfidf_vectorizer.transform(preprocessed_title)
    preds = dict()

    for modelo in modelos:
        preds[modelo] = modelo.predict(tfidf_vector)

    return preds

In [17]:
for i in df_externo[1]:
    print('-------------')
    print(predict_title(i))

-------------
{DecisionTreeClassifier(): array([1]), RandomForestClassifier(): array([1]), LogisticRegression(): array([0]), SVC(C=77, gamma=0.01): array([1])}
-------------
{DecisionTreeClassifier(): array([0]), RandomForestClassifier(): array([0]), LogisticRegression(): array([1]), SVC(C=77, gamma=0.01): array([0])}
-------------
{DecisionTreeClassifier(): array([1]), RandomForestClassifier(): array([1]), LogisticRegression(): array([1]), SVC(C=77, gamma=0.01): array([1])}
-------------
{DecisionTreeClassifier(): array([1]), RandomForestClassifier(): array([1]), LogisticRegression(): array([1]), SVC(C=77, gamma=0.01): array([1])}
-------------
{DecisionTreeClassifier(): array([1]), RandomForestClassifier(): array([1]), LogisticRegression(): array([0]), SVC(C=77, gamma=0.01): array([0])}
-------------
{DecisionTreeClassifier(): array([1]), RandomForestClassifier(): array([0]), LogisticRegression(): array([1]), SVC(C=77, gamma=0.01): array([1])}
-------------
{DecisionTreeClassifier():